In [1]:
# Import libraries
from pathlib import Path
import acoular as ac
import numpy as np
import pyfar as pf
import time
initialTime = time.time()

#---------------------------------------------------------------------------------------------------------------
# initial definitions
# name of result file
resFileName = "out/2025-01-28_15-59-01_400437_eval_test2"

# define microphone geometry
micgeofile = Path().resolve() / "in/array_position_data/bassoon_cage_64_optimized.xml"
m = ac.MicGeom(from_file=micgeofile)

# define calculation grid
resolution = 0.05
# g = ac.RectGrid3D(x_min=0.35, x_max=1.15, y_min=0.15, y_max=1.15, z_min=0.25, z_max=1.65, increment = resolution)
# g = ac.RectGrid3D(x_min=0.25, x_max=1.25, y_min=0.10, y_max=1.25, z_min=0.15, z_max=1.75, increment = resolution)
g = ac.RectGrid3D(x_min=0, x_max=1.2, y_min=0, y_max=1.2, z_min=0, z_max=1.75, increment=resolution)

# define audio data
name = 'in/array_audio_data/2025-01-28_15-59-01_400437.h5'
data = ac.MaskedTimeSamples(name = name)

# define time related variables
start = 0#83#26        # in s
stop = 105#35#92#36
framerate = 1           # in f/s

# define freqency resolution
bandwith = 1            # Octave = 1; 1/3 Octave = 3
frequency_bands = [125,250,500,1000,2000] # Array with mid frequencies of frequency bands

#---------------------------------------------------------------------------------------------------------------
# calculation of fixed values
# st = ac.SteeringVector(grid=g, mics=m, steer_type='true location')
frameLength = 1/framerate

print('init time: ' + str(time.time()-initialTime) + ' s')

result = np.zeros([len(frequency_bands),int((stop-start)*framerate),g.nxsteps,g.nysteps,g.nzsteps])


init time: 0.16579508781433105 s


In [2]:
# begin evaluation loop
calcTime = 0
i = 0
for i, currentFreqBand in enumerate(frequency_bands):

    frame = 0
    for frame in range(0,int((stop-start)*framerate)):
        calcTime = time.time()

        print('Frame length: ' + str((start+(frameLength*(frame+1)))-(start+(frameLength*frame))) + ' s ' + '(Frame ' + str(frame)+' of '+str((stop-start)*framerate)+')')
        print('Frequency Band: ' + str(i) + ' (' + str(currentFreqBand) + ' Hz)')

        # data = ac.MaskedTimeSamples(name = name)
        data = ac.MaskedTimeSamples(name = name, start = (start+(frameLength*frame))*data.sample_freq, stop = (start+(frameLength*(frame+1)))*data.sample_freq)
        # print('length of cropped measurement: ' + str(data.numsamples/data.sample_freq) + ' s (' + str(data.numsamples) + ' samples)')

        f = ac.PowerSpectra(source=data, window='Hanning', overlap='50%', block_size=2048)#256)#128)
        st = ac.SteeringVector(grid=g, mics=m, steer_type='true location')
        b = ac.BeamformerCleansc(freq_data=f, steer=st)

        result[i,frame] = b.synthetic(currentFreqBand, bandwith)

        print('Frame ' + str(frame) + ' calculated. Calculation time:' + str(time.time()-calcTime))
        frame = frame + 1
    
    i = i+1

Frame length: 1.0 s (Frame 0 of 105)
Frequency Band: 0 (125 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 1)]
[('2025-01-28_15-59-01_400437_cache.h5', 2)]
Frame 0 calculated. Calculation time:2.0505621433258057
Frame length: 1.0 s (Frame 1 of 105)
Frequency Band: 0 (125 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 3)]
[('2025-01-28_15-59-01_400437_cache.h5', 4)]
Frame 1 calculated. Calculation time:2.1149332523345947
Frame length: 1.0 s (Frame 2 of 105)
Frequency Band: 0 (125 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 5)]
[('2025-01-28_15-59-01_400437_cache.h5', 6)]


c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:170: RuntimeWarning: invalid value encountered in divide
  'true location': lambda x: x / sqrt(einsum('ij,ij->i', x, x.conj()) * x.shape[-1])[:, newaxis],
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:1750: RuntimeWarning: invalid value encountered in divide
  hh = (D1 + H * wmax) / sqrt(1 + dot(ww, H))
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:1746: RuntimeWarning: invalid value encountered in divide
  D1 = dot(csm.T - diag(diag(csm)), wmax) / hmax
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:170: RuntimeWarning: invalid value encountered in divide
  'true location': lambda x: x / sqrt(einsum('ij,ij->i', x, x.conj()) * x.shape[-1])[:, newaxis],
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:1750: RuntimeWarning: invalid value encountered in divide
  hh = (D1 + H * wmax) / sqrt(1 + dot(ww, H))
c:\Users\m

Frame 2 calculated. Calculation time:5.740206480026245
Frame length: 1.0 s (Frame 3 of 105)
Frequency Band: 0 (125 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 7)]
[('2025-01-28_15-59-01_400437_cache.h5', 8)]


c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:170: RuntimeWarning: invalid value encountered in divide
  'true location': lambda x: x / sqrt(einsum('ij,ij->i', x, x.conj()) * x.shape[-1])[:, newaxis],
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:1750: RuntimeWarning: invalid value encountered in divide
  hh = (D1 + H * wmax) / sqrt(1 + dot(ww, H))
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:1746: RuntimeWarning: invalid value encountered in divide
  D1 = dot(csm.T - diag(diag(csm)), wmax) / hmax
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:170: RuntimeWarning: invalid value encountered in divide
  'true location': lambda x: x / sqrt(einsum('ij,ij->i', x, x.conj()) * x.shape[-1])[:, newaxis],
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:1750: RuntimeWarning: invalid value encountered in divide
  hh = (D1 + H * wmax) / sqrt(1 + dot(ww, H))
c:\Users\m

Frame 3 calculated. Calculation time:5.5348875522613525
Frame length: 1.0 s (Frame 4 of 105)
Frequency Band: 0 (125 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 9)]
[('2025-01-28_15-59-01_400437_cache.h5', 10)]


...

Frame 6 calculated. Calculation time:9.530441522598267
Frame length: 1.0 s (Frame 7 of 105)
Frequency Band: 2 (500 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 435)]
[('2025-01-28_15-59-01_400437_cache.h5', 436)]


c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:170: RuntimeWarning: invalid value encountered in divide
  'true location': lambda x: x / sqrt(einsum('ij,ij->i', x, x.conj()) * x.shape[-1])[:, newaxis],
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:1750: RuntimeWarning: invalid value encountered in divide
  hh = (D1 + H * wmax) / sqrt(1 + dot(ww, H))
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:1746: RuntimeWarning: invalid value encountered in divide
  D1 = dot(csm.T - diag(diag(csm)), wmax) / hmax
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:170: RuntimeWarning: invalid value encountered in divide
  'true location': lambda x: x / sqrt(einsum('ij,ij->i', x, x.conj()) * x.shape[-1])[:, newaxis],
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:1750: RuntimeWarning: invalid value encountered in divide
  hh = (D1 + H * wmax) / sqrt(1 + dot(ww, H))
c:\Users\m

Frame 7 calculated. Calculation time:9.138758897781372
Frame length: 1.0 s (Frame 8 of 105)
Frequency Band: 2 (500 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 437)]
[('2025-01-28_15-59-01_400437_cache.h5', 438)]
Frame 8 calculated. Calculation time:8.586778163909912
Frame length: 1.0 s (Frame 9 of 105)
Frequency Band: 2 (500 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 439)]
[('2025-01-28_15-59-01_400437_cache.h5', 440)]
Frame 9 calculated. Calculation time:10.227762222290039
Frame length: 1.0 s (Frame 10 of 105)
Frequency Band: 2 (500 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 441)]
[('2025-01-28_15-59-01_400437_cache.h5', 442)]
Frame 10 calculated. Calculation time:8.238279104232788
Frame length: 1.0 s (Frame 11 of 105)
Frequency Band: 2 (500 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 443)]
[('2025-01-28_15-59-01_400437_cache.h5', 444)]
Frame 11 calculated. Calculation time:8.631792783737183
Frame length: 1.0 s (Frame 12 of 105)
Frequency Band: 2 (500 Hz)
[('2025-01-28_15-59-01_

c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:170: RuntimeWarning: invalid value encountered in divide
  'true location': lambda x: x / sqrt(einsum('ij,ij->i', x, x.conj()) * x.shape[-1])[:, newaxis],
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:1750: RuntimeWarning: invalid value encountered in divide
  hh = (D1 + H * wmax) / sqrt(1 + dot(ww, H))
c:\Users\moinm\anaconda3\envs\eds\Lib\site-packages\acoular\fbeamform.py:1746: RuntimeWarning: invalid value encountered in divide
  D1 = dot(csm.T - diag(diag(csm)), wmax) / hmax


Frame 59 calculated. Calculation time:12.832870721817017
Frame length: 1.0 s (Frame 60 of 105)
Frequency Band: 3 (1000 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 751)]
[('2025-01-28_15-59-01_400437_cache.h5', 752)]
Frame 60 calculated. Calculation time:20.95563006401062
Frame length: 1.0 s (Frame 61 of 105)
Frequency Band: 3 (1000 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 753)]
[('2025-01-28_15-59-01_400437_cache.h5', 754)]
Frame 61 calculated. Calculation time:21.52386236190796
Frame length: 1.0 s (Frame 62 of 105)
Frequency Band: 3 (1000 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 755)]
[('2025-01-28_15-59-01_400437_cache.h5', 756)]
Frame 62 calculated. Calculation time:19.87768578529358
Frame length: 1.0 s (Frame 63 of 105)
Frequency Band: 3 (1000 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 757)]
[('2025-01-28_15-59-01_400437_cache.h5', 758)]
Frame 63 calculated. Calculation time:21.646625995635986
Frame length: 1.0 s (Frame 64 of 105)
Frequency Band: 3 (1000 Hz)
[('2025-01-2

In [3]:
# save Data
np.save(resFileName,result)
pf.io.write(resFileName, 
           start = start, 
           stop = stop, 
           framerate = framerate, 
           frames = int((stop-start)*framerate), 
           frequency_bands = frequency_bands, 
           bandwith = bandwith,
           resolution = resolution,
           name = name)

In [4]:
# read Data

result = np.load(resFileName + ".npy")
start, stop, framerate, frames, frequency_bands, bandwith, resolution, name = pf.io.read(resFileName+".far")
result_Lp = ac.L_p(result)        # convert results in sound pressure Level